In [0]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.core.display import display, HTML

import folium



In [0]:
death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
country_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')

In [7]:
death_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1,1,2,4,4,4,4,4,4,4,6,6,7,7,11,14,14,15,15,18,18,21,23,25,30,30,30,33,36,36,40
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,2,2,2,2,2,4,5,5,6,8,10,10,11,15,15,16,17,20,20,21,22,22,23,23,23,23,23,24,25,26,26,26,26,26,26,27
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,3,4,4,4,7,9,11,15,17,17,19,21,25,26,29,31,35,44,58,86,105,130,152,173,193,205,235,256,275,293,313,326,336,348,364,367,375,384,392,402
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1,1,1,3,3,3,6,8,12,14,15,16,17,18,21,22,23,25,26,26,29,29,31,33,33,35,35,36,37,37,37
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2


In [0]:
confirmed_df.head()

In [0]:
recovered_df.head()

In [0]:
country_df.head()

In [0]:
country_df.columns=map(str.lower,country_df.columns)
death_df.columns=map(str.lower,death_df.columns)
recovered_df.columns=map(str.lower,recovered_df.columns)
confirmed_df.columns=map(str.lower,confirmed_df.columns)


In [0]:
confirmed_df = confirmed_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
recovered_df = recovered_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
death_df = death_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
country_df = country_df.rename(columns={'country_region': 'country'})

In [0]:
sorted_country_df = country_df.sort_values("confirmed",ascending=False).head(20)
sorted_country_df

In [71]:
def highlight_col(x):
    a = 'background-color:blue'
    b = 'background-color:red'
    c = 'background-color:orange'
    d = 'background-color:green'
    e = 'background-color:black'
    temp_df = pd.DataFrame('',index=x.index,columns=x.columns)
    temp_df.iloc[:,0]=a
    temp_df.iloc[:,4]=c
    temp_df.iloc[:,5]=b
    temp_df.iloc[:,6]=d
    temp_df.iloc[:,7]=e
    return temp_df

sorted_country_df.style.apply(highlight_col ,axis=None)

,country,last_update,lat,long_,confirmed,deaths,recovered,active,incident_rate,people_tested,people_hospitalized,mortality_rate,uid,iso3
17,US,2020-04-23 16:31:02,40.000000,-100.000000,845959,46972,77436,721551,256.766487,nan,nan,5.552515,840,USA
160,Spain,2020-04-23 16:30:31,40.463667,-3.749220,213024,22157,89250,101617,455.619696,nan,nan,10.401175,724,ESP
10,Italy,2020-04-23 16:30:31,41.871900,12.567400,189973,25549,57576,106848,314.203203,nan,nan,13.448753,380,ITA
6,France,2020-04-23 16:30:31,46.227600,2.213700,157135,21373,41331,94431,240.733178,nan,nan,13.601680,250,FRA
7,Germany,2020-04-23 16:30:31,51.165700,10.451500,151195,5354,103300,42541,180.458201,nan,nan,3.541122,276,DEU
16,United Kingdom,2020-04-23 16:30:31,55.000000,-3.000000,139243,18791,710,119742,205.112971,nan,nan,13.495113,826,GBR
172,Turkey,2020-04-23 16:30:31,38.963700,35.243300,98674,2376,16477,79821,116.996789,nan,nan,2.407929,792,TUR
89,Iran,2020-04-23 16:30:31,32.427908,53.688046,87026,5481,64843,16702,103.611073,nan,nan,6.298118,364,IRN
3,China,2020-04-23 15:26:19,30.592800,114.305500,83878,4636,77937,1305,5.971340,nan,nan,5.527075,156,CHN
13,Russia,2020-04-23 16:30:31,61.524000,105.318800,62773,555,4891,57327,43.014515,nan,nan,0.884138,643,RUS


In [72]:
fig = px.scatter(sorted_country_df.head(20),x='country',y='confirmed',size='confirmed',color='country',hover_name='country',size_max=60)
fig.show()

In [83]:
def plot_cases_for_country(country):
    labels = ['confirmed', 'deaths']
    colors = ['#667db6', '#e52d27']
    mode_size = [6,8]
    line_size = [4,5]

    df_list = [confirmed_df, death_df]

    fig = go.Figure()

    for i, df in enumerate(df_list):
        if country == 'World' or country == 'world':
            x_data = np.array(list(df.iloc[:, 5:].columns))
            y_data = np.sum(np.asarray(df.iloc[:, 5:]), axis=0)
        else:
            x_data = np.array(list(df.iloc[:, 5:].columns))
            y_data = np.sum(np.asarray(df[df['country']==country].iloc[:, 5:]),axis=0)
        
        fig.add_trace(go.Scatter(x=x_data, y=y_data, mode="lines+markers",name=labels[i],
                             line=dict(color=colors[i],width=line_size[i]),
                             connectgaps = True ,text='Total'+str(labels[i]) + ": "+ str(y_data[-1])
                                        ))
    
    fig.show()


plot_cases_for_country('India')
interact(plot_cases_for_country, country="World");


interactive(children=(Text(value='World', description='country'), Output()), _dom_classes=('widget-interact',)…

In [74]:
world_map = folium.Map(location=[11,0], tiles="cartodbpositron", zoom_start=2, max_zoom=5, min_zoom=2)

for i in range(len(confirmed_df)):
    folium.Circle(
    location=[confirmed_df.iloc[i]['lat'], confirmed_df.iloc[i]['long']],
    fill = True,
    radius = (int((np.log(confirmed_df.iloc[i,-1]+1.001))) + 0.7)*50000,
    fill_color= 'blue',
    color = 'red',
    tooltip = "<div style='margin: 0; background-color: red; color: #200122;'>"+
                    "<h4 style='text-align:center;font-weight: bold'>"+confirmed_df.iloc[i]['country'] + "</h4>"
                    "<hr style='margin:10px;color: white;'>"+
                    "<ul style='color: white;;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
                        "<li>Confirmed: "+str(confirmed_df.iloc[i,-1])+"</li>"+
                        "<li>Deaths:   "+str(death_df.iloc[i,-1])+"</li>"+
                        "<li>Death Rate: "+ str(np.round(death_df.iloc[i,-1]/(confirmed_df.iloc[i,-1]+1.00001)*100,2))+ "</li>"+
                    "</ul></div>",
    ).add_to(world_map)
    
    
world_map

In [75]:
px.bar(
    sorted_country_df.head(20),
    x = "country",
    y = "deaths",
    title= "Affected Cases",
    color_discrete_sequence=["#fc466b"], 
    height=750,
    width=1000
)


In [80]:
px.bar(
    sorted_country_df.head(20),
    x = "country",
    y = "confirmed",
    title= "confirmed Cases",
    color_discrete_sequence=["#fc466b"], 
    height=750,
    width=1000
)

In [82]:
px.bar(
    sorted_country_df.head(20),
    x = "country",
    y = "recovered",
    title= "confirmed Cases",
    color_discrete_sequence=["#fc466b"], 
    height=750,
    width=1000
)